### Manually define the schema

In [0]:
%sql
select current_catalog();

In [0]:

dbutils.fs.ls('/Volumes/workspace/default/lateflow_connect_volume')

In [0]:
landing_zone = '/Volumes/workspace/default/lateflow_connect_volume'
orders_data = landing_zone + '/orders_data'
checkpoint_path = landing_zone + '/manuallyInferschemacheckpoint'

In [0]:
from pyspark.sql.types import StructType, StringType,StructField,IntegerType,StringType,TimestampType

orders_schema = StructType([StructField("order_id", IntegerType(), True),
StructField("order_date", TimestampType(), True),
StructField("customer_id", IntegerType(), True),
StructField("order_status", StringType(), True)])

In [0]:
orders_df = (spark.readStream
.format("cloudFiles")
.option("cloudFiles.format", "csv")
.option("header", "true")
.option("rescuedDataColumn", "_rescued_data")
.schema(orders_schema)
.option("cloudFiles.schemaLocation", checkpoint_path)
.load(orders_data))

In [0]:
orders_df.printSchema();

In [0]:
%sql
drop table if exists ordersdeltastream;
create table ordersdeltastream;

In [0]:
(orders_df.writeStream
 .format("delta")
 .option("checkpointLocation", checkpoint_path)
 .option("mergeSchema", "true")
 .outputMode("append")
 .trigger(availableNow=True)
 .toTable("workspace.default.ordersdeltastream"))

In [0]:
%sql
select * from ordersdeltastream;

### upload files with mismatch schema. data will be ingested in _rescued_data column

In [0]:
(orders_df.writeStream
 .format("delta")
 .option("checkpointLocation", checkpoint_path)
 .option("mergeSchema", "true")
 .outputMode("append")
 .trigger(availableNow=True)
 .toTable("workspace.default.ordersdeltastream"))
 

In [0]:
%sql
select * from ordersdeltastream;

In [0]:
(orders_df.writeStream
 .format("delta")
 .option("checkpointLocation", checkpoint_path)
 .option("mergeSchema", "true")
 .outputMode("append")
 .trigger(availableNow=True)
 .toTable("workspace.default.ordersdeltastream"))
 

In [0]:
%sql
select * from ordersdeltastream;